Loading the requires packages

In [ ]:
library(twitteR)
library(ggplot2)
library("ggmap")
library("maptools")
library(maps)

Setting up the authentication and collecting tweets for a particular hashtag.

In [ ]:
setup_twitter_oauth("","","","")
hashtag = "#grammys"
tweets <- searchTwitter(hashtag, n= 2500, lang ="en")

Converting the recieved tweets into a dataframe and looking up user information using screen names. 

In [ ]:
tweet_data <- twListToDF(tweets)
user_info <- lookupUsers(tweet_data$screenName)


After converting the user info into a dataframe, fill the empty locations with NA and remove them.

In [ ]:
userDF <- twListToDF(user_info)
is.na(userDF$location) <- userDF$location == "" 
user_loc <- subset(userDF, !(is.na(userDF$location)),select=location) 

Find the geocode of these locations and remove the NA's from the resultant.

In [ ]:
locations <- geocode(user_loc$location)
locations <- subset(locations, !(is.na(locations$lon)),)


Write the recieved co-ordinates to a file and set append = TRUE for multiple iterations

In [ ]:
write.table(locations,"lat_lon.csv",append=TRUE, col.names=FALSE)

Read the csv file back and convert it into a dataframe with proper column names for further processing

In [116]:
loc <-read.table("lat_lon.csv")
lat <- loc$V2
lon <- loc$V3
loc<- data.frame(lat,lon)

Finding the state of each point using spatial data.[1]


Using USA states shapefile to determine the state of each co-ordinate point. This can be done using the readShapePoly() function which is a part of the maptools package. CRS(Co-ordinate Reference System) defines where the spatial objects are located on the earths surface. This is given as an input to proj4.

In [118]:
variableA=CRS("+init=epsg:4326 +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs +towgs84=0,0,0")
states=readShapePoly("US_states/states.shp",proj4string=variableA,verbose=TRUE)

Shapefile type: Polygon, (5), # of Shapes: 51


Coverts the location co-ordinates into spatial points. This is done so that the over() function can be sed to determine the state in which the co-ordinate lies.

In [119]:
points_sppol <- SpatialPoints(loc, proj4string=variableA)

Using the over() function to determine in which state polygon each of the points lie. This returns a points_in data frame which gives the state_name for each corresponding point


In [120]:
points_in <- over(points_sppol, states)

Removing the columns in state_name with NA entries 

In [121]:
location_US <- na.omit(points_in$STATE_NAME)

Converting the locations obtained into a table, which also gives the frequency of each state

In [123]:
table <- table(location_US)
table <- as.data.frame(table)

Get the geocodes of these returned states to plot them.

In [ ]:
geocodes <- geocode(as.character(table$location_US))

Plot these points on the map using frequency to plot the size of the blob.

In [ ]:
USA_loc <- data.frame(table[,1:2],geocodes)
map<-get_map(location="united states of america",zoom=4)
ggmap(map)+geom_point(data=USA_loc,aes(x=lon, y=lat, size=Freq),color="red")

[1] https://www.r-bloggers.com/r-and-gis-working-with-shapefiles/